In [1]:
# As usual, importing the libraries we need
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr

In [2]:
# Clean the corona data, a copy-paste from exercise 02
corona_df = pd.read_csv("../data/raw/corona/be_corona.csv", sep = "\t")

with open("../data/raw/metadata/be_metadata.json", 'r') as f:
   country_metadata = json.load(f)

region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}
corona_df["region"] = corona_df["PROVINCE"].map(region_map)

corona_df

,DATE,PROVINCE,CASES,region
0,2020-03-01,Antwerpen,1,BE-VAN
1,2020-03-01,Brussels,6,BE-BRU
2,2020-03-01,Limburg,1,BE-VLI
3,2020-03-01,Liège,3,BE-WLG
4,2020-03-01,OostVlaanderen,1,BE-VOV
...,...,...,...,...
3884,2021-02-22,Namur,90,BE-WNA
3885,2021-02-22,OostVlaanderen,206,BE-VOV
3886,2021-02-22,VlaamsBrabant,121,BE-VBR
3887,2021-02-22,WestVlaanderen,191,BE-VWV


In [3]:
# Clean the weather data, a copy-paste from exercise 01
weather_df = pd.read_csv("../data/raw/weather/weather.csv", sep = "\t")

weather_df["TemperatureAboveGround"] = weather_df["TemperatureAboveGround"] - 273.15
weather_df = weather_df[weather_df["iso3166-2"].str.startswith("BE")]

weather_df

,date,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
0,2020-02-13,BE-BRU,83.322794,1.824475e+06,2.391444e+06,6.574019,0.007057,4.000000,5.969118
1,2020-02-13,BE-VAN,85.186553,1.466610e+06,2.400277e+06,6.374715,0.008026,3.727273,5.007786
2,2020-02-13,BE-VBR,82.931599,1.647796e+06,2.391336e+06,6.543842,0.007092,4.020528,5.988184
3,2020-02-13,BE-VLI,84.205190,1.283450e+06,2.387647e+06,5.933657,0.007081,3.895954,5.428035
4,2020-02-13,BE-VOV,84.199671,2.000195e+06,2.399247e+06,6.827424,0.009497,3.851508,6.241541
...,...,...,...,...,...,...,...,...,...
17881,2020-11-14,BE-WBR,80.681880,3.144057e+06,2.404415e+06,12.921149,0.001145,4.151163,5.214244
17882,2020-11-14,BE-WHT,82.900329,2.657214e+06,2.403134e+06,12.796432,0.001104,3.994737,5.333816
17883,2020-11-14,BE-WLG,82.941558,3.311402e+06,2.353943e+06,11.950912,0.000300,4.265421,5.179618
17884,2020-11-14,BE-WLX,86.802651,3.450874e+06,2.337415e+06,10.673603,0.000024,3.749169,3.784905


In [4]:
# Merging corona and weather data
df = corona_df.merge(weather_df, left_on = ["DATE", "region"], right_on = ["date", "iso3166-2"])
df = df.drop(["DATE", "PROVINCE", "region"], axis = 1)

# Lost because the weather data only covers until Nov 14th, 2020
print(corona_df.shape[0] - df.shape[0])
# Lost because the corona data starts in Mar 1st, 2020
print(weather_df.shape[0] - df.shape[0])

df

1101
248


,CASES,date,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
0,1,2020-03-01,BE-VAN,71.268604,5.654529e+06,2.377257e+06,7.028275,0.000283,6.305556,5.901631
1,6,2020-03-01,BE-BRU,71.161766,4.395659e+06,2.369199e+06,6.913354,0.000604,6.088235,6.268015
2,1,2020-03-01,BE-VLI,73.492234,5.385017e+06,2.364463e+06,6.840184,0.002273,5.945087,6.277830
3,3,2020-03-01,BE-WLG,80.041325,4.336106e+06,2.303546e+06,4.735473,0.009123,5.766355,6.309143
4,1,2020-03-01,BE-VOV,75.257202,4.937132e+06,2.376949e+06,6.555772,0.000332,6.529002,6.814743
...,...,...,...,...,...,...,...,...,...,...
2783,111,2020-11-14,BE-WLX,86.802651,3.450874e+06,2.337415e+06,10.673603,0.000024,3.749169,3.784905
2784,158,2020-11-14,BE-WNA,81.910109,3.525320e+06,2.372435e+06,12.064028,0.000574,4.026616,4.847877
2785,294,2020-11-14,BE-VOV,85.082627,1.990421e+06,2.425690e+06,13.012328,0.001301,3.213457,5.063612
2786,124,2020-11-14,BE-VBR,81.428593,2.899614e+06,2.418138e+06,13.214198,0.001101,3.923754,4.696823


In [5]:
# This list contains the name of the variables we want to use as predictors
Xs = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed']

# Since we are running a lot of tests, we need to correct for multiple hypothesis testing
# This is the Bonferroni correction. If we run N tests and we determine our significance
# threshold as a p-value of 0.001, then we need to make sure that the p-values we look at
# are lower than 0.001 / N. Here I multiply the length of Xs by three, because we are
# running three correlations per variable (linear, Spearman, and log).
significance_threshold = 0.001 / (len(Xs) * 3)

# We start from linear correlation. We print the name of the variable, the correlation
# coefficient and whether the correlation is significant.
corrs = []
pvalues = []
for var in Xs:
    corr, pvalue = pearsonr(df["CASES"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")


RelativeHumiditySurface
0.296	2.5266227091231114e-57	True

SolarRadiation
-0.343	8.641068976641513e-78	True

Surfacepressure
-0.024	0.20983435511137594	False

TemperatureAboveGround
-0.196	1.2851556338728101e-25	True

Totalprecipitation
0.069	0.0002826113694777965	False

UVIndex
-0.454	1.2217025557699472e-141	True

WindSpeed
0.169	2.6245799879671344e-19	True



In [6]:
# Now we do the same for the Spearman rank correlation
for var in Xs:
    corr, pvalue = spearmanr(df["CASES"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

RelativeHumiditySurface
0.206	3.8119690660337426e-28	True

SolarRadiation
-0.290	5.3677075561948535e-55	True

Surfacepressure
0.186	4.020695292405173e-23	True

TemperatureAboveGround
-0.298	2.2462080578828497e-58	True

Totalprecipitation
-0.054	0.00419696117687709	False

UVIndex
-0.591	2.5306183812256882e-262	True

WindSpeed
0.050	0.007874113069876027	False



In [7]:
# And a thir dtime, for a log-transformed Pearson correlation
for var in Xs:
    corr, pvalue = pearsonr(np.log(df["CASES"]), df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")


RelativeHumiditySurface
0.210	4.537552001175853e-29	True

SolarRadiation
-0.312	4.138333550915235e-64	True

Surfacepressure
0.167	7.370980831339112e-19	True

TemperatureAboveGround
-0.253	6.745454684544101e-42	True

Totalprecipitation
-0.002	0.9242285151061563	False

UVIndex
-0.591	2.1544310960566976e-262	True

WindSpeed
0.084	8.646388376739648e-06	True



In [70]:
# We now prepare for running a multivariate linear regresion using statsmodel
# The library requires us to create a constant variable, to calculate the
# intercept.
df = sm.add_constant(df)
Xs.append("const")

In [71]:
# First we run the linear multivariate regression
est = sm.OLS(df["CASES"], df[Xs], hasconst = True).fit()
# Lots to unpack here, but let's focus on the basics. The R-squared (top-right)
# is a measure of prediction quality: how much of the daily variation in number
# of cases can we explain? The "P>|t|" column tells you the (non Bonferroni
# corrected) p-values of each variable *when keeping all the other constant*.
# For instance, this regression tells us that varying SolarRadiation doesn't
# tell us anything interesting if everything else is held constant.
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                  CASES   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     121.2
Date:                Wed, 24 Feb 2021   Prob (F-statistic):          9.21e-156
Time:                        15:08:46   Log-Likelihood:                -20581.
No. Observations:                2788   AIC:                         4.118e+04
Df Residuals:                    2780   BIC:                         4.122e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
RelativeHumiditySurface   

In [73]:
# We now log-transform the number of cases
est = sm.OLS(np.log(df["CASES"]), df[Xs], hasconst = True).fit()
# And many things changes, but for now let's only focus on the fact that the
# R-squared has much improved, i.e. this model is more powerful.
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                  CASES   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     353.4
Date:                Wed, 24 Feb 2021   Prob (F-statistic):               0.00
Time:                        15:08:59   Log-Likelihood:                -4714.7
No. Observations:                2788   AIC:                             9445.
Df Residuals:                    2780   BIC:                             9493.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
RelativeHumiditySurface   